# MACDの買いサインから上がる確率を検証
https://qiita.com/MandT500/items/52f253bffb14306611c6
# pandas_datareader -> yfinance


In [1]:
import matplotlib
matplotlib.matplotlib_fname()

import datetime
import numpy as np
import pandas as pd
import yfinance as yf
import seaborn as sns
import matplotlib.pyplot as plt
import time,sys,os
import joblib
today = datetime.date.today()

prod = '1y'
dys = 365
dates = '2021-06-27' # 用いるデータの入ったフォルダ

last = today +datetime.timedelta(days=-dys)
last = pd.to_datetime(last)

os.makedirs('./csv/'+str(today),exist_ok = True)

import requests

def send_line_notify(notification_message):
    """
    LINEに通知する
    """
    line_notify_token = 'NFuqiryl8uSLNdzCG4B0S0z9qTql7u8LCBP0aQCiJ90'
    line_notify_api = 'https://notify-api.line.me/api/notify'
    headers = {'Authorization': f'Bearer {line_notify_token}'}
    data = {'message': f'message: {notification_message}'}
    requests.post(line_notify_api, headers = headers, data = data)

In [2]:
"""TOPIX リストの読み込み"""
Core30 = pd.read_csv('../model01_JPX_yfinance/csv/JPX_data/TOPIX_Core30_20210608.csv')
T100 = pd.read_csv('../model01_JPX_yfinance/csv/JPX_data/TOPIX_100_20210618.csv')
T400 = pd.read_csv('../model01_JPX_yfinance/csv/JPX_data/TOPIX_400_20210619.csv')
#S500 = pd.read_csv('../model01_JPX_yfinance/csv/JPX_data/TOPIX_S500_20210618.csv')
S1200 = pd.read_csv('../model01_JPX_yfinance/csv/JPX_data/TOPIX_S1200_20210620.csv')

S500 = pd.read_csv('../model01_JPX_yfinance/csv/JPX_data/TOPIX_S500_with_info_20210620.csv'\
                       ,).drop('日付',axis=1)

data =pd.DataFrame(S500).copy() 
#data = data.iloc[5:15] # n10

data['code'] = [str(s)+".T" for s in data.code]
stocks = data['code']

#stocks = [str(s)+".T" for s in data.code]
#stocks.append("^N225")
tickers = yf.Tickers(" ".join(stocks))
stocks

0      1377.T
1      1379.T
2      1417.T
3      1662.T
4      1719.T
        ...  
488    9945.T
489    9946.T
490    9948.T
491    9956.T
492    9997.T
Name: code, Length: 493, dtype: object

# read CSV

In [3]:
closes= pd.read_csv('./csv/'+str(dates)+'/ttl_closes_'\
                    +str(len(stocks))+'_'+str(prod)+'.csv',index_col = 0)

In [4]:
closes

,1377.T,1379.T,1417.T,1662.T,1719.T,1720.T,1766.T,1786.T,1813.T,1815.T,...,9882.T,9889.T,9928.T,9934.T,9936.T,9945.T,9946.T,9948.T,9956.T,9997.T
Date,,,,,,,,,,,,,,,,,,,,,
2020-06-29,3403.442871,1983.212646,1559.128418,1740.585693,582.200073,547.089722,6865.095215,NaN,1366.706299,1984.041626,...,1453.665405,1441.996948,2241.946533,2294.779541,5901.297852,1768.581421,1489.880127,2109.508545,2037.158081,605.248901
2020-06-30,3408.397217,1974.458862,1571.836182,1760.055420,600.514404,549.057678,6874.875000,NaN,1380.315063,1985.966064,...,1442.004883,1445.875854,2220.332520,2316.965088,5901.297852,1764.644775,1489.880127,2124.212158,2062.671631,597.362915
2020-07-01,3284.545410,1942.361694,1544.465820,1752.267456,596.658752,536.265991,6796.640137,NaN,1375.454834,1933.045410,...,1408.967041,1391.570557,2191.841553,2279.345947,5724.259277,1725.277344,1444.493042,2095.784668,2037.158081,600.320129
2020-07-02,3254.821045,1934.580566,1521.983154,1771.737183,597.622620,537.250000,6845.536621,NaN,1360.874023,1929.196655,...,1396.334961,1373.145630,2171.210205,2317.929688,5802.942871,1723.308960,1402.066040,2115.389893,2065.615479,620.035034
2020-07-03,3225.096680,1925.826904,1532.735718,1781.471924,591.839233,526.426270,6747.743164,NaN,1375.454834,1926.310059,...,1448.806885,1386.721924,2166.297852,2327.575684,5743.930176,1720.356445,1403.052734,2203.612793,2053.840088,638.764282
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-22,3755.000000,1918.000000,1941.000000,1995.000000,827.000000,781.000000,10240.000000,283.0,1856.000000,1873.000000,...,1934.000000,1432.000000,1599.000000,2586.000000,5650.000000,2038.000000,1416.000000,2474.000000,2295.000000,962.000000
2021-06-23,3750.000000,1914.000000,1913.000000,1995.000000,811.000000,781.000000,10080.000000,282.0,1836.000000,1854.000000,...,1934.000000,1432.000000,1599.000000,2586.000000,5650.000000,2038.000000,1416.000000,2474.000000,2295.000000,962.000000
2021-06-24,3735.000000,1924.000000,1910.000000,2042.000000,812.000000,773.000000,10090.000000,282.0,1844.000000,1855.000000,...,1934.000000,1432.000000,1599.000000,2586.000000,5650.000000,2038.000000,1416.000000,2474.000000,2295.000000,962.000000


In [5]:
df = closes.copy()
# result data frame 作成
day = [1,3,5,10]
col = ['g_p_count']
period = [[5,25],[5,50],[5,75]] #,[20,50],[20,75]]

short = 6 #12
long = 19 # 26
signal = 4

for d in day:
    col.append('roc_d'+str(d)+'_plus')
result = pd.DataFrame(data=0,index=(stocks),columns=col)

for c,e in enumerate(df.columns):
    #MACD https://qiita.com/SaturdayInThePark/items/4b0a6bae1e81a157dbee
    df[str(e)+'_macd'] = df[e].ewm(span=short).mean().round(2) - \
                            df[e].ewm(span=long).mean().round(2)
    
    df[str(e)+'_signal'] = df[str(e)+'_macd'].ewm(span=signal).mean().round(2)
    df[str(e)+'_diff'] = df[str(e)+'_macd'] -  df[str(e)+'_signal']
    df[str(e)+'_g_point'] = False
    
    df[str(e)+'_macd_line'] = np.nan
    df[str(e)+'_signal_line'] = np.nan
    df[str(e)+'_macd_slop'] = np.nan
    df[str(e)+'_signal_slop'] = np.nan
    df[str(e)+'_line_deg'] =np.nan
    
    for d in day:
        df[str(e)+'_roc_d'+str(d)] =np.nan
        df[str(e)+'_roc_d'+str(d)+'_point'] = np.nan
    
    # ゴールデンクロスのタイミング取得
    for i in range(len(df.index)):
        if(df[str(e)+'_macd'][i-1] <  df[str(e)+'_signal'][i-1] and \
           df[str(e)+'_macd'][i] >  df[str(e)+'_signal'][i]):
            df[str(e)+'_g_point'].iat[i] = True
        
        #ゴールデンクロス時の傾き/角度を算出    
        a1 = df[str(e)+'_macd'][i] - df[str(e)+'_macd'][i-1-1]
        b1 = df[str(e)+'_macd'][i] - a1 * df[str(e)][i]
        df[str(e)+'_macd_slop'].iat[i] = a1
        
        a2 = df[str(e)+'_signal'][i] - df[str(e)+'_signal'][i-2]
        b2 = df[str(e)+'_signal'][i] - a2 * df[str(e)][i]
        df[str(e)+'_signal_slop'].iat[i] = a2
        df[str(e)+'_line_deg'].iat[i] = np.rad2deg(np.arctan(abs((a1-a2)/(1+a1*a2))))
        
        for l in range(7):
            x = len(df[str(e)][:i])-1+l-3
            if(x<= len(df[str(e)])-1):
                df[str(e)+'_macd_line'].iat[x] = a1*x+b1
                df[str(e)+'_signal_line'].iat[x] = a2*x+b2
        
        '''除くゴールデンクロスを選択'''
        if(df[str(e)+'_line_deg'].iat[i] < 60):
            df[str(e)+'_g_point'].iat[i] = False
        
        #傾きの条件設定
        if(not(df[str(e)+'_macd'].iat[i] > -df[str(e)].iat[i] *0.01)):
            df[str(e)+'_g_point'].iat[i] = False
        
        #〇日後に上昇しているか確認
        for d in day:           
            if(len(df[str(e)][:i])+d <= len(df[str(e)])):
                df[str(e)+'_roc_d'+str(d)].iat[i-1]= (df[str(e)][len(df[str(e)][:i])+d-1]\
                                            - df[str(e)][i-1])/df[str(e)][i] * 100
            else:
                df[str(e)+'_roc_d'+str(d)].iat[i-1] = (df[str(e)][-1]\
                                            - df[str(e)][i-1])/df[str(e)][i] * 100
            
            if(df[str(e)+'_roc_d'+str(d)].iat[i-1]>0):
                df[str(e)+'_roc_d'+str(d)+'_point'].iat[i-1] =1
            else:
                df[str(e)+'_roc_d'+str(d)+'_point'].iat[i-1] =0   
        #上昇していたらカウンタと加算

    #df.dropna(inplace=True)
    df.fillna(0,inplace=True)

    if(len(df[str(e)+'_g_point'])>0):
        result['g_p_count'].iat[c] += sum(df[str(e)+'_g_point'])
        for d in day:
            result['roc_d'+str(d)+'_plus'].iat[c] += sum(df[str(e)+'_roc_d'+str(d)+'_point']*df[str(e)+'_g_point'])
            #result['roc_d'+str(d)+'_plus'].iat[c] += df[str(e)+'_roc_d'+str(d)+'_point'].iat[c]
print(result)

C:\Users\kmtk49\Anaconda3\envs\TF2-GPU\lib\site-packages\ipykernel_launcher.py:68: RuntimeWarning: divide by zero encountered in double_scalars
C:\Users\kmtk49\Anaconda3\envs\TF2-GPU\lib\site-packages\ipykernel_launcher.py:68: RuntimeWarning: invalid value encountered in double_scalars


        g_p_count  roc_d1_plus  roc_d3_plus  roc_d5_plus  roc_d10_plus
code                                                                  
1377.T          1            1            1            1             1
1379.T          6            2            3            4             6
1417.T          3            2            2            1             2
1662.T          4            2            1            1             0
1719.T          3            2            1            1             1
...           ...          ...          ...          ...           ...
9945.T          5            3            2            2             3
9946.T          1            0            1            0             1
9948.T          4            3            2            1             0
9956.T          2            0            0            1             1
9997.T          2            0            1            1             0

[493 rows x 5 columns]


In [6]:
result.to_csv('./csv/'+str(today)+'/g_point_result_deg60_n'\
                    +str(len(stocks))+'_'+str(prod)+'_s'+str(short)+'_l'+str(long)+'.csv')

In [7]:
'''
_df = pd.DataFrame()
for i in range(len(df)):
    for j in range(len(df.columns)):
        if df.iat[i,j]==True:
            print(df.index[i],df.columns[j],df.iat[i,j])
'''        

'\n_df = pd.DataFrame()\nfor i in range(len(df)):\n    for j in range(len(df.columns)):\n        if df.iat[i,j]==True:\n            print(df.index[i],df.columns[j],df.iat[i,j])\n'

In [8]:
df

,1377.T,1379.T,1417.T,1662.T,1719.T,1720.T,1766.T,1786.T,1813.T,1815.T,...,9997.T_signal_slop,9997.T_line_deg,9997.T_roc_d1,9997.T_roc_d1_point,9997.T_roc_d3,9997.T_roc_d3_point,9997.T_roc_d5,9997.T_roc_d5_point,9997.T_roc_d10,9997.T_roc_d10_point
Date,,,,,,,,,,,,,,,,,,,,,
2020-06-29,3403.442871,1983.212646,1559.128418,1740.585693,582.200073,547.089722,6865.095215,0.0,1366.706299,1984.041626,...,55.68,0.045732,-1.320133,0.0,2.475234,1.0,8.580856,1.0,5.280523,1.0
2020-06-30,3408.397217,1974.458862,1571.836182,1760.055420,600.514404,549.057678,6874.875000,0.0,1380.315063,1985.966064,...,52.36,0.108372,0.492606,1.0,6.896548,1.0,5.582918,1.0,8.374377,1.0
2020-07-01,3284.545410,1942.361694,1544.465820,1752.267456,596.658752,536.265991,6796.640137,0.0,1375.454834,1933.045410,...,-0.30,0.524192,3.179644,1.0,9.062008,1.0,4.451515,1.0,12.241661,1.0
2020-07-02,3254.821045,1934.580566,1521.983154,1771.737183,597.622620,537.250000,6845.536621,0.0,1360.874023,1929.196655,...,0.88,23.188878,2.932106,1.0,1.697537,1.0,1.234579,1.0,14.814829,1.0
2020-07-03,3225.096680,1925.826904,1532.735718,1781.471924,591.839233,526.426270,6747.743164,0.0,1375.454834,1926.310059,...,2.60,9.363801,2.702705,1.0,-1.651649,0.0,-1.951949,0.0,10.810812,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-22,3755.000000,1918.000000,1941.000000,1995.000000,827.000000,781.000000,10240.000000,283.0,1856.000000,1873.000000,...,-13.56,1.704646,0.000000,0.0,0.000000,0.0,0.727651,1.0,0.727651,1.0
2021-06-23,3750.000000,1914.000000,1913.000000,1995.000000,811.000000,781.000000,10080.000000,282.0,1836.000000,1854.000000,...,-6.79,24.952040,0.000000,0.0,0.727651,1.0,0.727651,1.0,0.727651,1.0
2021-06-24,3735.000000,1924.000000,1910.000000,2042.000000,812.000000,773.000000,10090.000000,282.0,1844.000000,1855.000000,...,-1.80,39.039564,0.000000,0.0,0.727651,1.0,0.727651,1.0,0.727651,1.0


In [9]:

df.to_csv('./csv/'+str(today)+'/g_point_deg60_n'\
                    +str(len(stocks))+'_'+str(prod)+'_s'+str(short)+'_l'+str(long)+'.csv')


In [10]:
gp = pd.DataFrame(data=result.sum())
gp.columns =['sum'] 
gp

,sum
g_p_count,1507
roc_d1_plus,746
roc_d3_plus,729
roc_d5_plus,775
roc_d10_plus,783


In [11]:
print('roc_d1_plus:',gp.iloc[1]/gp.iloc[0],
     'roc_d3_plus:',gp.iloc[2]/gp.iloc[0],
     'roc_d5_plus:',gp.iloc[3]/gp.iloc[0],
     'roc_d10_plus:',gp.iloc[4]/gp.iloc[0],)

roc_d1_plus: sum    0.495023
dtype: float64 roc_d3_plus: sum    0.483743
dtype: float64 roc_d5_plus: sum    0.514267
dtype: float64 roc_d10_plus: sum    0.519575
dtype: float64
